In [1]:
import pandas as pd
import os
import numpy as np
import scanpy as sc

In /Users/sanjavickovic/miniconda2/envs/stenv3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/sanjavickovic/miniconda2/envs/stenv3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/sanjavickovic/miniconda2/envs/stenv3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/sanjavickovic/miniconda2/envs/stenv3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

Rename annotations in RAW anndata files

In [2]:
# list of already processed files
h5ad_path = '/Volumes/Uppmax/colon_data/raw_data'
data_ready = [i.split(".h5ad")[0] for i in os.listdir(h5ad_path)]
g = data_ready
#g = metadata[metadata.Age == '12w']['Filename'] # use to subest to a particular condition

In [4]:
# reformat the df_all 
#df_anns_subset = df_anns_all[df_anns_all['image'].isin(g)]
df_anns_subset = pd.read_csv(os.path.join("/Users/sanjavickovic/Desktop/colons_comp/splotch_ann/", "splotch_annotations_merged.tsv"), sep = "\t")
df_anns_subset['file_names'] = [str(i)+'_'+str(j) for i,j in zip(df_anns_subset['image'],df_anns_subset['x_y'])]
df_anns_subset.index = df_anns_subset['file_names']
df_anns_subset = df_anns_subset[['value']]

In [5]:
df_anns_subset

value
file_names                                                                 
L8CN183_C2_18.82_11.996   epithelium and lamina propria and muscularis m...
L8CN183_C2_3.624_15.187   epithelium and lamina propria and muscularis m...
L8CN183_C2_7.68_22.927    epithelium and lamina propria and muscularis m...
L8CN183_C2_14.736_27.978                      epithelium and lamina propria
L8CN183_C2_16.764_29.933  muscularis propria muscularis externa and musc...
...                                                                     ...
L8CN174_D1_9.03_15.037    epithelium and lamina propria and muscularis m...
L8CN174_D1_9.939_25.038                       epithelium and lamina propria
L8CN174_D1_21.99_19.055                       epithelium and lamina propria
L8CN174_D1_31.127_8.054                       epithelium and lamina propria
L8CN174_D1_9.89_22.124                        epithelium and lamina propria

[71900 rows x 1 columns]

Re-process original h5ad raw files to have same annotations

In [50]:
## this step is only done if h5ad raw counts files already exhist
h5ad_path_reprocessed = '/Volumes/Uppmax/colon_data/raw_data_reprocessed' # new output dir
files_done = [i.split(".h5ad")[0] for i in os.listdir(h5ad_path_reprocessed)] #check if any files hae already been re-processed
# g = ['10005CN35_C2']
for f in g:
    
    # check if already processed
    if f in files_done:
        continue 
    
    if f == '.DS_Store':
        continue
        
    filename = f + ".h5ad"
    p = sc.read_h5ad(os.path.join(h5ad_path, filename))
    adata_n = p.copy()   
    obs_tmp = pd.merge(p.obs, df_anns_subset, left_index = True, right_index = True).drop(['annotation'], axis = 1).rename(columns={"value": "annotation"})   
    adata_n = adata_n[obs_tmp.index]
    adata_n.obs = adata_n.obs.loc[np.unique(obs_tmp.index),:]
    adata_n.obs = obs_tmp
    adata_n.write_h5ad(filename = os.path.join(h5ad_path_reprocessed, f+'.h5ad'))

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'patch' as categorical
... storing 'centroids' as categorical
... storing 'annotation' as categorical
